You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [4]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [5]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [7]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [8]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [10]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [11]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [12]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [13]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [14]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Title: The Future of Artificial Intelligence: Trends, Players, and News

Introduction:
- Brief overview of Artificial Intelligence (AI) and its significance in today's tech-driven world
- Mention of recent advancements in AI technology and its impact on various industries

Key Points:
1. Latest Trends in Artificial Intelligence:
- Advances in machine learning algorithms
- Expansion of AI applications in healthcare, finance, and transportation
- Growth of AI-powered virtual assistants and chatbots
- Rise of aut

I now can give a great answer

Final Answer:
# The Future of Artificial Intelligence: Trends, Players, and News

**Introduction:**

Artificial Intelligence (AI) has become a pivotal aspect of our tech-driven world, revolutionizing industries and shaping the way we interact with technology. Recent advancements in AI technology have propelled innovation to new heights, with machine learning algorithms and AI applications making significant strides in various sectors. The impact of AI on healthcare, finance, transportation, and other industries has been profound, paving the way for a future powered by intelligent systems.

**Latest Trends in Artificial Intelligence:**

The continuous evolution of machine learning algorithms has been a driving force behind the latest trends in AI. From predictive analytics to personalized recommendations, AI has enhanced decision-making processes and streamlined operations across industries. The expansion of AI applications in healthcare, finance, and tran

- Display the results of your execution as markdown in the notebook.

In [15]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Trends, Players, and News

**Introduction:**

Artificial Intelligence (AI) has become a pivotal aspect of our tech-driven world, revolutionizing industries and shaping the way we interact with technology. Recent advancements in AI technology have propelled innovation to new heights, with machine learning algorithms and AI applications making significant strides in various sectors. The impact of AI on healthcare, finance, transportation, and other industries has been profound, paving the way for a future powered by intelligent systems.

**Latest Trends in Artificial Intelligence:**

The continuous evolution of machine learning algorithms has been a driving force behind the latest trends in AI. From predictive analytics to personalized recommendations, AI has enhanced decision-making processes and streamlined operations across industries. The expansion of AI applications in healthcare, finance, and transportation has led to improved efficiency and outcomes, showcasing the potential of intelligent systems in transforming traditional practices. Moreover, the growth of AI-powered virtual assistants and chatbots has revolutionized customer service and communication, providing personalized experiences and efficient solutions. The rise of autonomous vehicles and drones further demonstrates the capabilities of AI in revolutionizing transportation and logistics.

**Key Players in the AI Industry:**

Several key players have emerged as leaders in the AI industry, driving innovation and shaping the future of technology. Google's DeepMind has been at the forefront of AI research, pushing boundaries and unlocking new possibilities in machine learning and artificial intelligence. IBM Watson's cognitive computing capabilities have revolutionized industries such as healthcare and finance, showcasing the power of AI in data analysis and decision-making. Amazon Web Services (AWS) and Microsoft Azure have provided robust AI platforms and services, enabling businesses to leverage AI solutions for enhanced productivity and efficiency. Tesla's Autopilot technology has redefined the automotive industry, paving the way for autonomous driving and intelligent transportation systems.

**Noteworthy News in Artificial Intelligence:**

Recent breakthroughs in natural language processing have revolutionized communication and interaction with AI systems, enabling seamless conversations and personalized experiences. Ethical concerns surrounding AI and data privacy have sparked debates and discussions on the responsible use of intelligent systems, highlighting the importance of ethical considerations in AI development. Collaboration between tech giants and research institutions has led to groundbreaking discoveries and advancements in AI technology, fostering a culture of innovation and knowledge sharing. The impact of AI on the job market and workforce automation has raised questions about the future of work and the role of humans in an AI-driven world, prompting discussions on reskilling and upskilling in the digital age.

**Conclusion:**

In conclusion, the future of Artificial Intelligence holds immense potential for innovation and transformation across industries. By staying informed on AI trends and news, individuals can harness the power of intelligent systems to drive progress and create impactful solutions. It is essential to engage in discussions and explore the limitless possibilities of AI technology, shaping a future where intelligent systems enhance our lives and drive positive change. Embracing the advancements in AI and staying ahead of the curve will empower individuals to navigate the evolving landscape of technology and contribute to a future powered by artificial intelligence.

Join the conversation and explore the limitless possibilities of AI technology!

*Remember to subscribe to industry newsletters, engage in discussions on social media platforms, and explore online courses or workshops on AI technology to stay informed and empowered in the era of Artificial Intelligence.*

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [16]:
topic = "KS Statistics"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on KS Statistics.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: KS Statistics

Title: Understanding KS Statistics: Trends, Players, and How It Impacts You

Outline:
I. Introduction
- Brief explanation of KS Statistics and its significance
- Mention of recent trends and key players in the field

II. Latest Trends in KS Statistics
- Overview of the current landscape
- Analysis of emerging technologies and methodologies
- Case studies highlighting successful implementations

III. Key Players in KS Statistics
- Profiles of leading companies and organizations
- Interviews

I now can give a great answer 

Final Answer: 

# Understanding KS Statistics: Trends, Players, and How It Impacts You

## Introduction
KS Statistics, also known as Kolmogorov-Smirnov statistics, is a powerful tool used in data analysis to determine if a sample comes from a specific distribution. Its significance lies in its ability to test the goodness of fit between observed data and a theoretical distribution. Recent trends in KS Statistics have shown a growing interest in advanced statistical methods and data interpretation techniques. Key players in the field include leading companies and organizations that specialize in data analysis and research.

## Latest Trends in KS Statistics
The current landscape of KS Statistics is evolving with the introduction of emerging technologies and methodologies. Data scientists and statisticians are exploring innovative approaches to analyze complex datasets and make informed decisions. Case studies showcasing successful implementations of KS St

In [17]:
Markdown(result)

# Understanding KS Statistics: Trends, Players, and How It Impacts You

## Introduction
KS Statistics, also known as Kolmogorov-Smirnov statistics, is a powerful tool used in data analysis to determine if a sample comes from a specific distribution. Its significance lies in its ability to test the goodness of fit between observed data and a theoretical distribution. Recent trends in KS Statistics have shown a growing interest in advanced statistical methods and data interpretation techniques. Key players in the field include leading companies and organizations that specialize in data analysis and research.

## Latest Trends in KS Statistics
The current landscape of KS Statistics is evolving with the introduction of emerging technologies and methodologies. Data scientists and statisticians are exploring innovative approaches to analyze complex datasets and make informed decisions. Case studies showcasing successful implementations of KS Statistics highlight its practical applications in various industries, from finance to healthcare.

## Key Players in KS Statistics
Profiles of leading companies and organizations in the field of KS Statistics demonstrate their expertise in statistical analysis techniques and data interpretation methods. Interviews with experts provide valuable insights into different approaches and strategies for applying KS Statistics in real-world scenarios. By comparing these diverse perspectives, professionals can gain a comprehensive understanding of the best practices in the field.

## Noteworthy News in KS Statistics
Recent developments and innovations in KS Statistics are shaping the future of data analysis and research. Insights from industry events and conferences offer valuable perspectives on the latest trends and advancements in the field. Future predictions and forecasts for KS Statistics point towards a continued growth in demand for advanced statistical methods and data research techniques.

## Target Audience Analysis
Professionals in data analysis, research, and statistics form the target audience for content on KS Statistics. Their interests lie in advanced statistical methods and data interpretation techniques that can enhance their decision-making processes. Addressing the pain points of complex data interpretation and providing practical solutions will resonate with this audience and add value to their work.

## SEO Keywords
- KS Statistics
- Statistical analysis techniques
- Data interpretation methods
- Key statistical players
- Advanced data research trends

## Resources
For further learning on KS Statistics, research papers, articles, and industry reports are valuable sources of information. Online courses and webinars offer opportunities to deepen one's understanding of statistical analysis techniques and methodologies. Relevant websites and forums provide platforms for ongoing discussions and knowledge sharing in the field of data interpretation.

## Call to Action
I encourage readers to explore the world of KS Statistics and delve deeper into the realm of statistical analysis and data interpretation. By providing links to additional resources for in-depth knowledge, readers can continue their learning journey and stay updated on the latest trends in the field. I invite feedback and comments for future content ideas to ensure that the blog remains a valuable resource for professionals in data analysis and statistics.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).